In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
source_root = '../../data/classification_direction'
train_root = '../../data/classification_direction/train'
val_root = '../../data/classification_direction/val'

In [3]:
# import os
# import random
# import shutil
# from torchvision.datasets import ImageFolder

# def create_dir_structure(root_path, classes):
#     for cls in classes:
#         os.makedirs(os.path.join(root_path, cls), exist_ok=True)

# def copy_files(file_paths, source_root, target_root):
#     for file_path in file_paths:
#         class_name = os.path.basename(os.path.dirname(file_path))
#         target_path = os.path.join(target_root, class_name, os.path.basename(file_path))
#         shutil.copy2(file_path, target_path)

# # Загрузка исходного датасета
# dataset = ImageFolder(root=source_root)
# class_names = dataset.classes

# # Определим количество данных для тренировки и валидации
# train_ratio = 0.8
# total_size = len(dataset)
# train_size = int(total_size * train_ratio)
# val_size = total_size - train_size

# # Перемешаем данные и разделим их
# indices = list(range(total_size))
# random.shuffle(indices)
# train_indices, val_indices = indices[:train_size], indices[train_size:]

# train_files = [dataset.imgs[idx][0] for idx in train_indices]
# val_files = [dataset.imgs[idx][0] for idx in val_indices]

# # Создадим структуру папок для тренировочных и валидационных данных
# create_dir_structure(train_root, class_names)
# create_dir_structure(val_root, class_names)

# # Скопируем файлы в соответствующие папки
# copy_files(train_files, source_root, train_root)
# copy_files(val_files, source_root, val_root)


In [4]:
batch_size = 32
num_epochs = 3
learning_rate = 0.0005

In [5]:
# Трансформации для данных

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.04),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# Загрузка данных
train_dataset = datasets.ImageFolder(root=train_root, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [7]:
val_dataset = datasets.ImageFolder(root=val_root, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# Загрузка модели MobileNet
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)  # Заменяем последний слой для бинарной классификации

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Использование GPU, если доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [10]:
# Функция потерь и оптимизатор
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=num_epochs)

In [11]:
# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        # scheduler.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            predicted = torch.sigmoid(outputs).round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

print('Обучение завершено')

Epoch 1/3, Loss: 0.3807
Validation Loss: 0.1014, Accuracy: 0.9510
Epoch 2/3, Loss: 0.0439
Validation Loss: 0.0494, Accuracy: 1.0000
Epoch 3/3, Loss: 0.0169
Validation Loss: 0.0130, Accuracy: 1.0000
Обучение завершено


In [13]:
import onnx

dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(model, dummy_input, "mobilenet_v2.onnx", input_names=["input"], output_names=["output"], opset_version=11)

In [14]:
import onnxruntime as ort
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

class ImageClassifierONNX:
    def __init__(self, onnx_path):
        self.onnx_session = ort.InferenceSession(onnx_path)
        self.input_name = self.onnx_session.get_inputs()[0].name
        self.output_name = self.onnx_session.get_outputs()[0].name
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image_path):
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image).unsqueeze(0).numpy()
        outputs = self.onnx_session.run([self.output_name], {self.input_name: image})
        output = outputs[0][0][0]
        return output

classifier_onnx = ImageClassifierONNX('mobilenet_v2.onnx')
for path in os.listdir('../../data/classification_direction/val/bad/'):
  result_onnx = classifier_onnx.predict(f'../../data/classification_direction/val/bad/{path}')
  print(f'Результат классификации: {result_onnx:.4f}')

print()
for path in os.listdir('../../data/classification_direction/val/good/'):
  result_onnx = classifier_onnx.predict(f'../../data/classification_direction/val/good/{path}')
  print(f'Результат классификации: {result_onnx:.4f}')